This notebook will run a SHARPy case of a primsatic canitilever wing. The wing has a chord of 1m and an aspect ratio of 16, the structual properties are arbitrary. <br> 
The main variables that will vary are: <br>

>    Density: 0.05529 (20km) -> 1.01325 (sea level) <br>
     Setting Angle: 0 -> 5 (degrees)<br>
     Gust Length:  chord 1 -> 10 chord lengths (V_inf = 5, W_o = 0.1) <br> 
     
The data we want to store is the freestream velocity, position and velocites at each node.

## Import Packages

In [1]:
#load normal Python Packages
import numpy as np                         # basic mathematical and array functions
import os                                  # Functions related to the operating system
import matplotlib.pyplot as plt            # Plotting library

#load specicfic methods from SHARPy
import sharpy.sharpy_main                  # Run SHARPy inside jupyter notebooks
import sharpy.utils.plotutils as pu        # Plotting utilities
import sharpy.utils.generate_cases as gc   # Automate model generation.
import sharpy.utils.algebra as algebra

#set seed for random operations
SEED = 1
np.random.seed(SEED)

## Specify Problem Paramters

In [2]:
#Wing Geometry
chord = 1.         # Chord of the wing
aspect_ratio = 16. # Ratio between lenght and chord: aspect_ratio = length/chord
wake_length = 25   # Length of the wake in chord lengths, should be large enough to ensure convergance
sweep_angle = 0    # Wing sweep in rad, positive backwards.

# Discretization
num_node = 11           # Number of nodes in the structural discretisation
num_chord_panels = 5    # Number of aerodynamic panels in the chordwise direction
num_points_camber = 20  # The camber line of the wing will be defined by a series of (x,y)
                        # coordintes. Here, we define the size of the (x,y) vectors

# Structural properties of the beam cross section
mass_per_unit_length = 0.75 # Mass per unit length
mass_iner_x = 0.1           # Mass inertia around the local x axis
mass_iner_y = 0.001         # Mass inertia around the local y axis
mass_iner_z = 0.001         # Mass inertia around the local z axis
pos_cg_B = np.zeros((3))    # position of the centre of mass with respect to the elastic axis
EA = 1e6                    # Axial stiffness
GAy = 1e6                   # Shear stiffness in the local y axis
GAz = 1e6                   # Shear stiffness in the local z axis
GJ = 1e4                    # Torsional stiffness
EIy = 2e4                   # Bending stiffness around the flapwise direction
EIz = 5e6                   # Bending stiffness around the edgewise direction

# Flight Condition
Vinf_initial = 5.       # Wind speed at start of simulation
beta = 0.*np.pi/180.     # Sideslip in rad
gust_intensity = 0.5     # Gust intensity

# Time discretization
dt = chord / (num_chord_panels * Vinf_initial)  # One timesteps per panel
end_time = 4.0                                  # End time of the simulation
time_steps = int(end_time/dt)+1                 # Number of timesteps required in simulation


In [3]:
# Parameters that will randomly be varied
num_tests = 200                                    # Number of simulations that wil be run
air_density_range = (0.05529, 1.01325)             # Range of air densities (sea level to 20km)
gust_length_range = (1.*chord, 10.*chord)          # Range of gust lengths   
initial_AoA_range = (0.*np.pi/180, 5.*np.pi/180)   # Range of wing setting angles

# Create arrays of random values for each simulation:
air_densities = np.random.uniform(*air_density_range, num_tests)
gust_lengths  = np.random.uniform(*gust_length_range, num_tests)
initial_AoAs  = np.random.uniform(*initial_AoA_range, num_tests)


## Iterate Through Cases
I am aware that iterating through like this is far from the best way of doing it. But resetting up SHARPy on each iteration was the easiest way to get it working reliably so I could get data and move onto the machine learning side of things.

In [4]:
last_test = 0

In [5]:
for test in range(last_test, num_tests):
    
    last_test = test
    
    #set new gust length
    gust_length = gust_lengths[test]
    #set new density
    air_density = air_densities[test]
    #set new intial AoA
    initial_AoA = initial_AoAs[test]
    
    ### First initialise the SHARPy simulation ###
    
    SimInfo = gc.SimulationInformation()                   # Initialises the SimulationInformation class
    SimInfo.set_default_values()                           # Assigns the default values to all the solvers
    OutputDir = 'output_' + str(test) + '/'
    if not os.path.exists(OutputDir):
        os.mkdir(OutputDir)
    SimInfo.solvers['SHARPy']['route'] = OutputDir         # Folder where outputs are written.
    SimInfo.solvers['SHARPy']['write_screen'] = 'off'
    wing = gc.AeroelasticInformation()                     # Initialize aeroelastic model.
    
    ### Then Define the structual properties of the wing
    
    wing.StructuralInformation.num_node = num_node         # Define the number of nodes
    wing.StructuralInformation.num_node_elem = 3           # Define the number of nodes per element
    wing.StructuralInformation.compute_basic_num_elem()    # Compute the number of elements assuming basic connections
    node_r = np.zeros((num_node, 3))                               # Generate an array with the location of the nodes
    node_r[:,1] = np.linspace(0, chord*aspect_ratio, num_node)
    wing.StructuralInformation.generate_uniform_beam(node_r,       # Send inputs to model generation function to produce a finite-element model of the wing
                                                     mass_per_unit_length,
                                                     mass_iner_x, mass_iner_y, mass_iner_z,
                                                     pos_cg_B,
                                                     EA, GAy, GAz,
                                                     GJ, EIy, EIz,
                                                     num_node_elem = wing.StructuralInformation.num_node_elem,
                                                     y_BFoR = 'x_AFoR',
                                                     num_lumped_mass=0)
    wing.StructuralInformation.frame_of_reference_delta *= -1
    SimInfo.solvers['BeamLoader']['unsteady'] = 'on'              # Define structural simulation parameters.
    SimInfo.solvers['BeamLoader']['orientation'] = algebra.euler2quat(np.array([0.0,initial_AoA,beta]))
    
    ### Then Define the aerodynamic properties
    
    wake_panels = int(wake_length*chord/dt)                            # Compute the number of panels in the wake (streamwise direction) based on the previous parameter
    airfoil_camber = np.zeros((1, num_points_camber, 2))               # Define the coordinates of the camber line of the airfoils used. Here it is a flat plate.
    airfoil_camber[0, :, 0] = np.linspace(0, 1, num_points_camber)
    wing.AerodynamicInformation.create_one_uniform_aerodynamics (wing.StructuralInformation,  # Send inputs to model generation function to produce a vortex-lattice model of the wing.
                                                                 chord = chord,
                                                                 twist = 0.,
                                                                 sweep = sweep_angle,                
                                                                 num_chord_panels = num_chord_panels,
                                                                 m_distribution = 'uniform',
                                                                 elastic_axis = 0.5,                 # E.A. at mid chord
                                                                 num_points_camber = num_points_camber,
                                                                 airfoil = airfoil_camber)
    
    ### Apply Boundary Conditions and Mirror Wing

    wing2 = wing.copy()
    wing2.StructuralInformation.coordinates -= wing.StructuralInformation.coordinates[-1, :]
    wing.assembly(wing2)
    wing.remove_duplicated_points(1e-9)
    wing.StructuralInformation.boundary_conditions[0] = 1 # Clamped
    wing.StructuralInformation.boundary_conditions[num_node] = -1 # Free end (right wing)
    wing.StructuralInformation.boundary_conditions[num_node + 1] = -1 # Free end (left wing)
    
    ### Setup Aerodynamic Solvers
    
    SimInfo.solvers['AerogridLoader']['unsteady'] = 'on'     # Define aerodynamic simulation parameters. Initialize a wake along the global X axis.
    SimInfo.solvers['AerogridLoader']['aligned_grid'] = 'on'
    SimInfo.solvers['AerogridLoader']['mstar'] = wake_panels
    SimInfo.solvers['AerogridLoader']['freestream_dir'] = np.array([1.,0.,0.])
    SimInfo.solvers['AerogridLoader']['wake_shape_generator'] = 'StraightWake'
    SimInfo.solvers['AerogridLoader']['wake_shape_generator_input'] = {'u_inf': Vinf_initial, 
                                                                       'u_inf_direction' : np.array([1., 0., 0.]),
                                                                       'dt' : dt}
    SimInfo.set_variable_all_dicts('rho', air_density)
    
    ### Setup Static Simulation

    SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                                         'AerogridLoader',
                                         'StaticCoupled',
                                         'BeamPlot',
                                         'AerogridPlot']
    SimInfo.solvers['SHARPy']['case'] = 'case_static'
    SimInfo.solvers['StaticUvlm']['horseshoe'] = False    # Use finite wake as in the dynamic problem.
    SimInfo.solvers['StaticUvlm']['n_rollup'] = 0
    SimInfo.solvers['StaticUvlm']['velocity_field_generator'] = 'SteadyVelocityField'
    SimInfo.solvers['StaticUvlm']['velocity_field_input'] = {'u_inf': Vinf_initial,
                                                             'u_inf_direction': np.array([1.,0.,0.])}
    SimInfo.solvers['NonLinearStatic']['gravity_on'] = False   # Remove gravitational effects.
    SimInfo.solvers['NonLinearStatic']['max_iterations'] =150
    SimInfo.solvers['NonLinearStatic']['num_load_steps'] =1
    SimInfo.solvers['NonLinearStatic']['min_delta'] =1e-6
    SimInfo.solvers['NonLinearStatic']['gravity'] =9.754
    SimInfo.solvers['StaticCoupled']['structural_solver'] = 'NonLinearStatic'
    SimInfo.solvers['StaticCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearStatic']
    SimInfo.solvers['StaticCoupled']['aero_solver'] = 'StaticUvlm'
    SimInfo.solvers['StaticCoupled']['aero_solver_settings'] = SimInfo.solvers['StaticUvlm']
    SimInfo.solvers['StaticCoupled']['n_load_steps'] = 0
    SimInfo.solvers['StaticCoupled']['relaxation_factor'] = 0.0
    SimInfo.solvers['StaticUvlm']['rollup_dt'] = dt
    SimInfo.solvers['StaticCoupled']['tolerance'] = 1.e-6
    SimInfo.solvers['StaticCoupled']['max_iter'] = 50
    SimInfo.solvers['AeroForcesCalculator'] = {'write_text_file': 'on',
                                              'screen_output': 'off'}
    
    ### Setup Dynamic Simulation
    SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                                         'AerogridLoader',
                                         'StaticCoupled',
                                         'AeroForcesCalculator',
                                         'DynamicCoupled']
    SimInfo.solvers['SHARPy']['case'] = 'case_dynamic'
    SimInfo.set_variable_all_dicts('dt', dt)
    SimInfo.define_num_steps(time_steps)
    SimInfo.solvers['StepUvlm']['convection_scheme'] = 2  
    SimInfo.solvers['StepUvlm']['velocity_field_generator'] = 'GustVelocityField'
    SimInfo.solvers['StepUvlm']['velocity_field_input'] = {'u_inf' : Vinf_initial,
                                                           'u_inf_direction': np.array([1., 0., 0.]),
                                                           'relative_motion': 'on',
                                                           'gust_shape': '1-cos',
                                                           'gust_parameters': {'gust_length': gust_length,
                                                                               'gust_intensity':gust_intensity}}
    SimInfo.solvers['NonLinearDynamicPrescribedStep']['gravity_on'] = False
    SimInfo.solvers['DynamicCoupled']['structural_solver'] = 'NonLinearDynamicPrescribedStep'
    SimInfo.solvers['DynamicCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearDynamicPrescribedStep']
    SimInfo.solvers['DynamicCoupled']['aero_solver'] = 'StepUvlm'
    SimInfo.solvers['DynamicCoupled']['aero_solver_settings'] = SimInfo.solvers['StepUvlm']
    SimInfo.solvers['DynamicCoupled']['postprocessors'] = ['BeamPlot', 'AerogridPlot', 'AeroForcesCalculator', 'WriteVariablesTime']
    SimInfo.solvers['DynamicCoupled']['postprocessors_settings'] = {'BeamPlot': SimInfo.solvers['BeamPlot'],
                                                                    'AerogridPlot': SimInfo.solvers['AerogridPlot'],
                                                                    'AeroForcesCalculator': SimInfo.solvers['AeroForcesCalculator'],
                                                                    'WriteVariablesTime': {'structure_nodes': num_node,  # free end node_id
                                                                                            'structure_variables': ['pos'],
                                                                                            'cleanup_old_solution': 'on', }, }
    SimInfo.solvers['DynamicCoupled']['include_unsteady_force_contribution'] = True
    
    
    ### Run Simulation
    
    gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
    wing.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
    SimInfo.generate_solver_file()
    SimInfo.generate_dyn_file(time_steps)
    
    ### run SHARPy
    sharpy_output = sharpy.sharpy_main.main(['',
                                         SimInfo.solvers['SHARPy']['route'] +
                                         SimInfo.solvers['SHARPy']['case'] +
                                         '.sharpy'])
    
    ### Post Process

    ### First collect data from the Sharpy_Output file
    #setup array to hold SHARPY data and fill in
    SHARPy_Output_Data = np.zeros([time_steps,5])
    SHARPy_Output_Data[:, 0] = np.arange(time_steps)                                   #Step
    SHARPy_Output_Data[:, 1] = np.linspace(0, end_time, time_steps, dtype='float64')   #Time
    SHARPy_Output_Data[:, 2] = Vinf_initial                                            #Velocity
    SHARPy_Output_Data[:, 3] = air_density                                             #Density
    SHARPy_Output_Data[:, 4] = initial_AoA                                             #Angle
    # add gust to Velocity at each point
    for i in range(time_steps):
        time = SHARPy_Output_Data[i, 1]
        if time <= (gust_length / Vinf_initial):
            SHARPy_Output_Data[i, 2] = Vinf_initial + (gust_intensity / 2)*(1 - np.cos(2*np.pi*Vinf_initial*time / gust_length))
            
    ### Collect data from the sharpy_output dataset variable
    #setup arrays
    Aero_Forces   = np.zeros([time_steps, num_node*3])
    Positions     = np.zeros([time_steps, num_node*3])
    Velocities    = np.zeros([time_steps, num_node*3])
    Accelerations = np.zeros([time_steps, num_node*3])
    #iterate through each timestep
    for i in range(time_steps):
        # gather aero forces at each node
        SHARPy_Aero_Steady_Forces   = np.transpose(np.sum(sharpy_output.aero.timestep_info[i].forces[0][0:3,:,:], axis=1))
        SHARPy_Aero_Unsteady_Forces = np.transpose(np.sum(sharpy_output.aero.timestep_info[i].dynamic_forces[0][0:3,:,:], axis=1))
        SHARPy_Aero_Forces = SHARPy_Aero_Steady_Forces + SHARPy_Aero_Unsteady_Forces 
        # gather displacement, velocity and acceleration at each node
        SHARPy_Pos   = sharpy_output.structure.timestep_info[i].pos[:num_node,:]
        SHARPy_Vel   = sharpy_output.structure.timestep_info[i].pos_dot[:num_node,:]
        SHARPy_Accel = sharpy_output.structure.timestep_info[i].pos_ddot[:num_node,:]
        # write date by each node
        for j in range(num_node):
            Aero_Forces[i, j*3:(j+1)*3]   = SHARPy_Aero_Forces[j,:]
            Positions[i, j*3:(j+1)*3]     = SHARPy_Pos[j,:]
            Velocities[i, j*3:(j+1)*3]    = SHARPy_Vel[j,:]
            Accelerations[i, j*3:(j+1)*3] = SHARPy_Accel[j,:]
    
    ### Save files
    np.savetxt(OutputDir + "Main_SHARPy_Data.csv", SHARPy_Output_Data, fmt='%.18e', delimiter=',', newline='\n', header='Step, Time, Vinf, Density, AoA', footer='', comments='# ', encoding=None)
    np.savetxt(OutputDir + "Aero_Forces.csv",      Aero_Forces,        fmt='%.18e', delimiter=',', newline='\n', header='Fx1, Fy1, Fz1, ...',                         footer='', comments='# ', encoding=None)
    np.savetxt(OutputDir + "Positions.csv",        Positions,          fmt='%.18e', delimiter=',', newline='\n', header='x1, y1, z1, x2, y2, z2, ...',                            footer='', comments='# ', encoding=None)
    np.savetxt(OutputDir + "Velocities.csv",       Velocities,         fmt='%.18e', delimiter=',', newline='\n', header='x1, y1, z1, x2, y2, z2, ...',                            footer='', comments='# ', encoding=None)
    np.savetxt(OutputDir + "Accelerations.csv",    Accelerations,      fmt='%.18e', delimiter=',', newline='\n', header='x1, y1, z1, x2, y2, z2, ...',                            footer='', comments='# ', encoding=None)

fatal: not a git repository (or any of the parent directories): .git


                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   Res

fatal: not a git repository (or any of the parent directories): .git


                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   Res

fatal: not a git repository (or any of the parent directories): .git


...

                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi


fatal: not a git repository (or any of the parent directories): .git


                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi


fatal: not a git repository (or any of the parent directories): .git


                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   Res